<a href="https://colab.research.google.com/github/Durga22-amie/-vqe-cancer-segmentation-/blob/main/mask_and_vqe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 45.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.4.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [2]:
!pip install tensorflow numpy opencv-python tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.4.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.3 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
input_folder = "/content/drive/My Drive/dataset/YES NO Dataset"
output_folder = "/content/drive/My Drive/dataset/YES NO Dataset masks"


In [1]:
import os
import cv2
import numpy as np

# ---------------------------------------
# PATHS (CHANGE THESE)
# ---------------------------------------
input_folder = "/content/drive/My Drive/dataset/YES NO Dataset"
output_folder = "/content/drive/My Drive/dataset/YES NO Dataset masks"      # Folder where masks will be saved

# ---------------------------------------
# CREATE OUTPUT FOLDER IF NOT EXISTS
# ---------------------------------------
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# ---------------------------------------
# PROCESS EACH IMAGE
# ---------------------------------------
for filename in os.listdir(input_folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".tif", ".bmp")):

        # Full input path
        img_path = os.path.join(input_folder, filename)

        # Read image in grayscale
        img = cv2.imread(img_path, 0)

        if img is None:
            print(f"Skipping {filename}, cannot read image.")
            continue

        # ---------------------------------------
        # Apply threshold to create mask
        # You can change 120 to 100 / 150
        # ---------------------------------------
        _, mask = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)

        # Save mask
        mask_path = os.path.join(output_folder, f"mask_{filename}")
        cv2.imwrite(mask_path, mask)

        print(f"Saved mask: {mask_path}")

print("\nAll masks generated successfully!")


Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_8_yes.jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_4_yes.jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_6_yes.jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_5_yes.jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_9_yes (2).jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_3_yes.jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_2_yes.jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_1_yes.jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_10_no (2).jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_7_yes.jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_8_no.jpg
Saved mask: /content/drive/My Drive/dataset/YES NO Dataset masks/mask_5_no.jpg
Saved mask: /content/drive/My Driv

In [2]:
import os
import cv2
import numpy as np

input_folder = "/content/drive/My Drive/dataset/YES NO Dataset"
output_folder = "/content/drive/My Drive/dataset/YES NO Dataset_UNET_masks"

os.makedirs(output_folder, exist_ok=True)

def preprocess(img):
    img = cv2.resize(img, (256, 256))
    img = img / 255.0
    return img.reshape(1, 256, 256, 1)

for filename in os.listdir(input_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):

        path = os.path.join(input_folder, filename)

        # Read grayscale MRI
        img = cv2.imread(path, 0)
        if img is None:
            continue

        # Preprocess
        inp = preprocess(img)

        # Predict with U-Net
        pred = model.predict(inp)[0, :, :, 0]

        # Threshold output
        mask = (pred > 0.5).astype(np.uint8) * 255

        # Resize mask back to original size
        mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

        # Save mask
        out_path = os.path.join(output_folder, "mask_" + filename)
        cv2.imwrite(out_path, mask)

        print("Saved:", out_path)

print("\n🎉 All U-Net masks generated successfully!")


NameError: name 'model' is not defined

In [3]:
import glob
import numpy as np
import os

# Define the folder containing your masks
masks_directory_path = "/content/drive/My Drive/dataset/YES NO Dataset_UNET_masks"

# !!! CORRECTED LINE !!!
# Use os.path.join to build the search pattern for ALL .npy files inside the directory
search_pattern = os.path.join(masks_directory_path, "*.npy")
# If they are images, use "*.png" or "*.jpg"

list_of_masks = []
mask_files = glob.glob(search_pattern)

if not mask_files:
    print(f"Error: No files found using pattern: {search_pattern}. Check your path and file extensions.")
else:
    print(f"Found {len(mask_files)} files. Loading data...")
    for file_path in mask_files:
        mask_data = np.load(file_path)

        # Add validation and append to list
        if mask_data.shape == (218, 180):
            list_of_masks.append(mask_data)
        else:
            print(f"Skipping file {file_path} due to incorrect shape {mask_data.shape}")

# NOW PASTE THE REST OF THE VQE PROCESSING CODE HERE
# ... (the rest of the function definition and main processing loop from before) ...


Error: No files found using pattern: /content/drive/My Drive/dataset/YES NO Dataset_UNET_masks/*.npy. Check your path and file extensions.


In [4]:
import numpy as np
from sklearn.decomposition import PCA
from qiskit.circuit.library import pauli_feature_map, efficient_su2
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp

# --- CONFIGURATION ---
NUM_FEATURES = 8 # Number of features to reduce to via PCA (and number of qubits)
REPS_FEATURE_MAP = 1
REPS_ANSATZ = 2
MAX_VQE_ITERATIONS = 100
# ---------------------

# Assume 'list_of_masks' is a list containing your 20 U-Net NumPy arrays
# Example placeholder data (replace this with your actual mask loading code):
# list_of_masks = [np.random.rand(218, 180) for _ in range(20)]
# (Add your code here to load the real mask data)
list_of_masks = [] # Initialize your list here

def extract_vqe_feature(mask_2d: np.ndarray) -> float:
    """
    Processes a single (218, 180) mask through PCA and VQE to extract one feature value.
    """

    # 1. Classical Dimensionality Reduction (PCA)
    n_pixels = mask_2d.size
    flattened_data = mask_2d.reshape(1, n_pixels)

    pca = PCA(n_components=NUM_FEATURES)
    reduced_features = pca.fit_transform(flattened_data).flatten()

    # 2. Scale the data for angle encoding (0 to 2*pi range)
    if reduced_features.max() == reduced_features.min():
        scaled_input_vector = np.zeros(NUM_FEATURES)
    else:
        scaled_input_vector = (reduced_features - reduced_features.min()) / \
                              (reduced_features.max() - reduced_features.min()) * (2 * np.pi)

    # 3. Create the Quantum Circuit Components
    num_qubits = NUM_FEATURES
    feature_map = pauli_feature_map(feature_dimension=num_qubits, reps=REPS_FEATURE_MAP, entanglement='linear')
    ansatz = efficient_su2(num_qubits=num_qubits, reps=REPS_ANSATZ, entanglement='linear')
    observable = SparsePauliOp.from_list([('Z'*num_qubits, 1.0)])

    # 4. Bind the data and combine circuits
    encoded_circuit = feature_map.assign_parameters(scaled_input_vector)
    full_circuit = encoded_circuit.compose(ansatz)

    # 5. Setup and Run VQE
    estimator = StatevectorEstimator()
    optimizer = COBYLA(maxiter=MAX_VQE_ITERATIONS)
    vqe = VQE(estimator, ansatz, optimizer=optimizer)

    # Run VQE and capture the optimal value
    result = vqe.compute_minimum_eigenvalue(observable)

    return result.optimal_value

# --- Main loop to process all masks ---

vqe_features = []

print(f"Starting VQE feature extraction for {len(list_of_masks)} masks...")

for i, mask in enumerate(list_of_masks):
    print(f"Processing mask {i+1}/{len(list_of_masks)}...")
    try:
        feature_value = extract_vqe_feature(mask)
        vqe_features.append(feature_value)
        print(f"  -> Extracted feature: {feature_value}")
    except Exception as e:
        print(f"Error processing mask {i+1}: {e}")

print("\nFinished extraction.")
print(f"Collected {len(vqe_features)} VQE features.")
print(f"Features: {vqe_features}")

# Now you can save these features to a CSV or use them in your classical classifier
# np.savetxt("vqe_extracted_features.csv", vqe_features, delimiter=",")


Starting VQE feature extraction for 0 masks...

Finished extraction.
Collected 0 VQE features.
Features: []


In [5]:
from qiskit.circuit.library import efficient_su2
ansatz = efficient_su2(num_qubits=8, reps=3, entanglement="full")


In [6]:
def mask_to_hamiltonian(vec):
    terms = []
    for i, value in enumerate(vec):
        pauli_string = 'I'*i + 'Z' + 'I'*(8-i-1)
        weight = float(value)
        terms.append((pauli_string, weight))
    return SparsePauliOp.from_list(terms)


In [7]:
def load_masks(folder):
    files = sorted(os.listdir(folder))
    masks = []
    for f in files:
        if f.endswith(".png") or f.endswith(".jpg"):
            img = Image.open(os.path.join(folder, f)).convert("L")
            img = img.resize((16, 16))
            vec = np.array(img).flatten() / 255.0
            masks.append(vec)
    return masks

In [8]:
import cv2

mask_path = "/content/drive/My Drive/dataset/YES NO Dataset masks/mask_1_yes.jpg"

# Load as grayscale
mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

if mask_img is None:
    print("Error: mask image not found!")
else:
    print("Mask loaded:", mask_img.shape)


Mask loaded: (218, 180)


In [9]:
mask_16 = cv2.resize(mask_img, (16, 16))
vec256 = mask_16.flatten()

print("Vector length:", len(vec256))    # should be 256



Vector length: 256


In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# The original problem was trying to fit PCA with n_components=8 on a single sample.
# vec256.reshape(1, -1) results in (1 sample, 256 features).
# PCA requires n_components <= n_samples.

pca = PCA(n_components=8)

# To make PCA fit with n_components=8, we need a dataset with at least 8 samples.
# For demonstration, we'll create a small synthetic dataset based on vec256.
# In a real scenario, you would gather multiple 'vec256' arrays from different masks.
num_synthetic_samples = 10 # Must be >= n_components
synthetic_data = np.array([vec256 + np.random.normal(0, 5, vec256.shape) for _ in range(num_synthetic_samples)])

# Fit PCA on the synthetic dataset
pca.fit(synthetic_data)

# Then transform the original vec256 using the fitted PCA model
vec8 = pca.transform(vec256.reshape(1, -1))[0]

print("8-dimensional vector:", vec8)

In [ ]:
from qiskit.quantum_info import SparsePauliOp

terms = []
n_qubits = 8

for i, value in enumerate(vec8):
    pauli = 'I'*i + 'Z' + 'I'*(n_qubits-i-1)
    terms.append((pauli, float(value)))

H = SparsePauliOp.from_list(terms)
print(H)


In [ ]:
! pip install qiskit-aer

In [ ]:
!pip install --upgrade --force-reinstall qiskit qiskit-aer qiskit-algorithms

In [ ]:

from qiskit.circuit.library import efficient_su2
from qiskit.primitives import Estimator
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit.quantum_info import SparsePauliOp

# Example 8-qubit Hamiltonian
pauli_strings = ['ZIIIIIII', 'IZIIIIII', 'IIZIIIII', 'IIIZIIII',
                 'IIIIZIII', 'IIIIIZII', 'IIIIIIZI', 'IIIIIIIZ']
coeffs = [-0.6487, -2.6251, 0.9424, 0.0924, -0.2661, -3.2568, 0.9093, -4.3029]

# Add small X terms to avoid AlgorithmError
pauli_strings += ['XIIIIIII', 'IXIIIIII']
coeffs += [0.05, 0.05]

H = SparsePauliOp(pauli_strings, coeffs=coeffs)

# Ansatz
ansatz = efficient_su2(num_qubits=8, reps=3, entanglement='full')

# Estimator + optimizer
estimator = Estimator()
optimizer = SPSA(maxiter=50)

# VQE
vqe = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)

# Run
result = vqe.compute_minimum_eigenvalue(operator=H)
print("VQE Energy:", result.eigenvalue.real)

In [11]:
!pip install qiskit==0.41.0


  Using cached qiskit-0.41.0.tar.gz (14 kB)
  Preparing metadata (setup.py) ... done
  Using cached qiskit-terra-0.23.1.tar.gz (4.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached qiskit-aer-0.11.2.tar.gz (6.5 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [12]:
! pip install qiskit-algorithms

In [10]:
import numpy as np
# Import the functions instead of the classes
from qiskit.circuit.library import pauli_feature_map, efficient_su2
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp

# 1. Define your 8-dimensional data
input_vector = np.array([-0.64876986, -2.62517237, 0.94244815, 0.09240809,
                         -0.2661078, -3.25682053, 0.90935779, -4.30299167])

num_qubits = len(input_vector)

# 2. Scale the data for angle encoding
scaled_input_vector = (input_vector - input_vector.min()) / (input_vector.max() - input_vector.min()) * (2 * np.pi)

# 3. Create the Feature Map using the FUNCTION `pauli_feature_map`
# This function returns a QuantumCircuit object directly
feature_map = pauli_feature_map(feature_dimension=num_qubits, reps=1, entanglement='linear')
encoded_circuit = feature_map.assign_parameters(scaled_input_vector)

# 4. Create the Ansatz using the FUNCTION `efficient_su2`
ansatz = efficient_su2(num_qubits=num_qubits, reps=2, entanglement='linear')
full_circuit = encoded_circuit.compose(ansatz)

# 5. Define an Observable (Cost function/Hamiltonian)
observable = SparsePauliOp.from_list([('Z'*num_qubits, 1.0)])

# 6. Set up and Run VQE
estimator = StatevectorEstimator()
optimizer = COBYLA(maxiter=100)

vqe = VQE(estimator, ansatz, optimizer=optimizer)

# The observable is passed as the first POSITIONAL argument
result = vqe.compute_minimum_eigenvalue(observable, aux_operators=[])

print(f"VQE Optimal Value: {result.optimal_value}")


VQE Optimal Value: -0.7363146854558065
